In [1]:
import pandas as pd
import numpy as np

In [2]:
data_path = 'nii_inf/export_EPI_MAIN_weekly_201540_201939_FINAL.xlsx'
base_df = pd.read_excel(data_path)

In [3]:
base_df.head()

,REGION_NAME,DISTRICT_NAME,LPU_NAME,YEAR,WEEK,ARI_0_2,ARI_3_6,ARI_7_14,ARI_15,ARI_TOTAL,ILI_0_2,ILI_3_6,ILI_7_14,ILI_15,ILI_TOTAL,POP_0_2,POP_3_6,POP_7_14,POP_15,POP_TOTAL
0,Дальневосточный ФО,Биробиджан,ЦГиЭ,2015,40,212,163,87,85,547,0,0,0,0,0,2929,3810,6470,61568,74777
1,Дальневосточный ФО,Владивосток,ЦГиЭ,2015,40,631,811,359,570,2371,0,0,0,0,0,19634,26033,48036,495273,588976
2,Дальневосточный ФО,Магадан,ЦГиЭ,2015,40,149,200,155,128,632,0,0,0,0,0,3696,4601,8429,84319,101045
3,Дальневосточный ФО,Петропавловск,ЦГиЭ,2015,40,297,334,226,193,1050,0,0,0,0,0,6754,7979,13889,152393,181015
4,Дальневосточный ФО,Улан-Удэ,ЦГиЭ,2015,40,144,159,75,228,606,0,0,0,0,0,23314,28731,45498,365197,462740


In [4]:
len(set(base_df['DISTRICT_NAME']))

61

In [5]:
new_df = set(base_df['DISTRICT_NAME'])
lower_names = [name.lower() for name in new_df]

In [6]:
new_df = pd.DataFrame(lower_names, columns=['lower_city'])
new_df.head()

,lower_city
0,ярославль
1,архангельск
2,ульяновск
3,мурманск
4,вологда


In [7]:
base_df['lower_city'] = [name.lower() for name in base_df['DISTRICT_NAME']]

In [8]:
new_df = new_df.merge(base_df[['lower_city', 'POP_TOTAL']].drop_duplicates(subset='lower_city'), on='lower_city', how='inner')

In [9]:
len(new_df)

61

# air_quality

In [10]:
air_quality = pd.read_csv('new_data/air_polution.csv')
air_quality.head()

,Город,"Республика, область, край",2017 год,2018 год,2019 год
0,БЕЛГОРОД,Белгородская обл.,Н,Н,Н
1,БРЯНСК,Брянская обл.,Н,Н,Н
2,ВЛАДИМИР,Владимирская обл.,Н,Н,Н
3,ВОЛГОРЕЧЕНСК,Костромская обл.,Н,Н,Н
4,ВОРОНЕЖ,Воронежская обл.,В,В,П


### air quality in numbers

In [11]:
number_quality = air_quality[['2017 год', '2018 год', '2019 год']].replace(
    to_replace=['Н', 'П', 'В', 'ОВ'], value=[1, 2, 3, 4]
)

In [12]:
number_quality.columns = ['2017_quality', '2018_quality', '2019_quality']
number_quality.head()

,2017_quality,2018_quality,2019_quality
0,1,1,1
1,1,1,1
2,1,1,1
3,1,1,1
4,3,3,2


In [13]:
air_quality = pd.concat([air_quality, number_quality], axis=1)
air_quality.head(10)

,Город,"Республика, область, край",2017 год,2018 год,2019 год,2017_quality,2018_quality,2019_quality
0,БЕЛГОРОД,Белгородская обл.,Н,Н,Н,1,1,1
1,БРЯНСК,Брянская обл.,Н,Н,Н,1,1,1
2,ВЛАДИМИР,Владимирская обл.,Н,Н,Н,1,1,1
3,ВОЛГОРЕЧЕНСК,Костромская обл.,Н,Н,Н,1,1,1
4,ВОРОНЕЖ,Воронежская обл.,В,В,П,3,3,2
5,ВОСКРЕСЕНСК,Московская обл.,Н,Н,Н,1,1,1
6,ГУБКИН,Белгородская обл.,Н,Н,Н,1,1,1
7,ДЗЕРЖИНСКИЙ,Московская обл.,Н,Н,Н,1,1,1
8,ИВАНОВО,Ивановская обл.,П,П,П,2,2,2
9,КАЛУГА,Калужская обл.,Н,П,П,1,2,2


In [14]:
air_quality['lower_city'] = [name.lower() for name in air_quality['Город']]

In [15]:
new_df = new_df.merge(air_quality, on='lower_city', how='left', suffixes=('_1', '_2'))

In [16]:
new_df = new_df.drop(['Город', 'Республика, область, край'], axis=1)
new_df.head(3)

,lower_city,POP_TOTAL,2017 год,2018 год,2019 год,2017_quality,2018_quality,2019_quality
0,ярославль,591400,Н,Н,Н,1,1,1
1,архангельск,357409,Н,П,П,1,2,2
2,ульяновск,642263,Н,Н,П,1,1,2


In [17]:
len(new_df)

61

# city_area

In [18]:
city_area = pd.read_csv('new_data/city_area.csv')
city_area.head()

,Unnamed: 0,number,city,sqr_km
0,0,1,Москва,2551.900
1,1,2,Санкт-Петербург,1439.000
2,2,3,Волгоград,859.353
3,3,4,Пермь,801.440
4,4,5,Уфа,700.000


In [19]:
city_area['lower_city'] = [name.lower() for name in city_area['city']]

In [20]:
new_df = new_df.merge(city_area, on='lower_city', how='left', suffixes=('_1', '_2'))

In [21]:
new_df = new_df.drop(['Unnamed: 0', 'number'], axis=1)
new_df.head()

,lower_city,POP_TOTAL,2017 год,2018 год,2019 год,2017_quality,2018_quality,2019_quality,city,sqr_km
0,ярославль,591400,Н,Н,Н,1,1,1,Ярославль,205.80
1,архангельск,357409,Н,П,П,1,2,2,Архангельск,294.42
2,ульяновск,642263,Н,Н,П,1,1,2,Ульяновск,300.00
3,мурманск,309150,Н,Н,Н,1,1,1,Мурманск,154.60
4,вологда,316614,Н,Н,Н,1,1,1,Вологда,115.73


In [22]:
len(new_df)

61

# habbits

In [23]:
habbits = pd.read_csv('new_data/habbits.csv', index_col=0)
habbits.head()

,city,habbit_rate
0,Брянск,"99,2"
1,Волгоград,"81,4"
2,Саратов,"79,8"
3,Смоленск,79
4,Ставрополь,79


In [24]:
new_df = new_df.merge(habbits, on='city', how='left', suffixes=('_1', '_2'))
new_df.head()

,lower_city,POP_TOTAL,2017 год,2018 год,2019 год,2017_quality,2018_quality,2019_quality,city,sqr_km,habbit_rate
0,ярославль,591400,Н,Н,Н,1,1,1,Ярославль,205.80,"47,6"
1,архангельск,357409,Н,П,П,1,2,2,Архангельск,294.42,"55,5"
2,ульяновск,642263,Н,Н,П,1,1,2,Ульяновск,300.00,"73,6"
3,мурманск,309150,Н,Н,Н,1,1,1,Мурманск,154.60,"36,1"
4,вологда,316614,Н,Н,Н,1,1,1,Вологда,115.73,"55,6"


In [25]:
len(new_df)

61

# Money

In [26]:
money = pd.read_csv('new_data/money.csv', index_col=0)
money.head()

,city,perc_more_100k,percent_less_15k,most_common
0,Москва,"31,6","1,2",46–125
1,Магадан,"27,3","0,3",47–113
2,Южно-Сахалинск,"21,9","0,4",42–94
3,Киров,"18,9","1,3",38–89
4,Санкт-Петербург,"13,5","2,2",35–76


In [27]:
new_df = new_df.merge(money, on='city', how='left', suffixes=('_1', '_2'))
new_df.head()

,lower_city,POP_TOTAL,2017 год,2018 год,2019 год,2017_quality,2018_quality,2019_quality,city,sqr_km,habbit_rate,perc_more_100k,percent_less_15k,most_common
0,ярославль,591400,Н,Н,Н,1,1,1,Ярославль,205.80,"47,6","2,1","13,4",20–42
1,архангельск,357409,Н,П,П,1,2,2,Архангельск,294.42,"55,5","7,5","2,3",26–59
2,ульяновск,642263,Н,Н,П,1,1,2,Ульяновск,300.00,"73,6","1,6",17,18–36
3,мурманск,309150,Н,Н,Н,1,1,1,Мурманск,154.60,"36,1",13,"0,5",32–70
4,вологда,316614,Н,Н,Н,1,1,1,Вологда,115.73,"55,6","3,6",9,22–49


In [28]:
new_df.head()

,lower_city,POP_TOTAL,2017 год,2018 год,2019 год,2017_quality,2018_quality,2019_quality,city,sqr_km,habbit_rate,perc_more_100k,percent_less_15k,most_common
0,ярославль,591400,Н,Н,Н,1,1,1,Ярославль,205.80,"47,6","2,1","13,4",20–42
1,архангельск,357409,Н,П,П,1,2,2,Архангельск,294.42,"55,5","7,5","2,3",26–59
2,ульяновск,642263,Н,Н,П,1,1,2,Ульяновск,300.00,"73,6","1,6",17,18–36
3,мурманск,309150,Н,Н,Н,1,1,1,Мурманск,154.60,"36,1",13,"0,5",32–70
4,вологда,316614,Н,Н,Н,1,1,1,Вологда,115.73,"55,6","3,6",9,22–49


In [29]:
len(new_df)

61

# Temperature

In [30]:
temp_df = pd.read_csv('new_data/temperatures', index_col=0)

In [31]:
temp_df.head()

,"Средняя годовая температура, °C","Температура самого холодного месяца, °C","Температура трёх самых холодных месяцев, °C","Температура самого тёплого месяца, °C","Температура трёх самых тёплых месяцев, °C","Абсолютный минимум,","Абсолютный максимум, °C","Среднее годовое количество осадков, мм",city
3,"11,9","0,3","1,1",24,"22,9","−33,7",40,718,Краснодар
5,"10,5","−3,7","−3,1","25,6","24,2","−37,7",41,233,Астрахань
6,"9,9","−3,1","−2,6","23,3","22,3","−31,9","40,1",643,Ростов-на-Дону
8,"9,3","−2,9","−1,9",22,"20,9","−27,8","39,7",556,Ставрополь
9,"9,2","−1,9","−1,4","20,6","19,6","−27,8","39,2",929,Владикавказ


In [32]:
new_df = new_df.merge(temp_df, on='city', how='left', suffixes=('_1', '_2'))

In [33]:
new_df.head()

,lower_city,POP_TOTAL,2017 год,2018 год,2019 год,2017_quality,2018_quality,2019_quality,city,sqr_km,...,percent_less_15k,most_common,"Средняя годовая температура, °C","Температура самого холодного месяца, °C","Температура трёх самых холодных месяцев, °C","Температура самого тёплого месяца, °C","Температура трёх самых тёплых месяцев, °C","Абсолютный минимум,","Абсолютный максимум, °C","Среднее годовое количество осадков, мм"
0,ярославль,591400,Н,Н,Н,1,1,1,Ярославль,205.80,...,"13,4",20–42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,архангельск,357409,Н,П,П,1,2,2,Архангельск,294.42,...,"2,3",26–59,"1,3","−12,8","−11,3","16,3","14,1","−45,2","34,4",607.0
2,ульяновск,642263,Н,Н,П,1,1,2,Ульяновск,300.00,...,17,18–36,5,"−10,3","−9,5","20,2","18,8","−40,0","39,3",470.0
3,мурманск,309150,Н,Н,Н,1,1,1,Мурманск,154.60,...,"0,5",32–70,"0,6","−10,0","−9,3","12,9","11,1","−39,4","32,9",494.0
4,вологда,316614,Н,Н,Н,1,1,1,Вологда,115.73,...,9,22–49,"3,1","−10,8","−9,7","17,5","15,8","−47,1","36,4",565.0


In [34]:
new_df.shape

(61, 22)

# Доктора

In [35]:
temp_df = pd.read_csv('new_data/terapevts.csv', index_col=0)
new_df = new_df.merge(temp_df, on='city', how='left', suffixes=('_1', '_2'))

In [36]:
temp_df = pd.read_csv('new_data/pediators.csv', index_col=0)
new_df = new_df.merge(temp_df, on='city', how='left', suffixes=('_1', '_2'))

In [37]:
new_df

,lower_city,POP_TOTAL,2017 год,2018 год,2019 год,2017_quality,2018_quality,2019_quality,city,sqr_km,...,"Температура трёх самых холодных месяцев, °C","Температура самого тёплого месяца, °C","Температура трёх самых тёплых месяцев, °C","Абсолютный минимум,","Абсолютный максимум, °C","Среднее годовое количество осадков, мм",num_terapevts,num_terapevts_100k_peaple,num_pediators,num_pediators_100k_peaple
0,ярославль,591400,Н,Н,Н,1,1,1,Ярославль,205.800,...,NaN,NaN,NaN,NaN,NaN,NaN,650.0,6.34,539.0,22.47
1,архангельск,357409,Н,П,П,1,2,2,Архангельск,294.420,...,"−11,3","16,3","14,1","−45,2","34,4",607.0,NaN,NaN,NaN,NaN
2,ульяновск,642263,Н,Н,П,1,1,2,Ульяновск,300.000,...,"−9,5","20,2","18,8","−40,0","39,3",470.0,639.0,6.26,379.0,16.76
3,мурманск,309150,Н,Н,Н,1,1,1,Мурманск,154.600,...,"−9,3","12,9","11,1","−39,4","32,9",494.0,238.0,3.98,232.0,14.93
4,вологда,316614,Н,Н,Н,1,1,1,Вологда,115.730,...,"−9,7","17,5","15,8","−47,1","36,4",565.0,276.0,2.98,326.0,13.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,томск,578596,П,Н,Н,2,1,1,Томск,297.200,...,"−15,6","18,7","16,7","−55,0","35,1",568.0,568.0,6.67,456.0,20.15
57,владимир,350529,Н,Н,Н,1,1,1,Владимир,137.014,...,"−8,0",18,"17,2","−43,0","37,1",584.0,490.0,4.35,374.0,14.82
58,калининград,448548,Н,Н,Н,1,1,1,Калининград,223.000,...,"−1,0","18,1","17,1","−33,3","36,5",818.0,349.0,4.36,261.0,13.50
59,челябинск,1183387,В,П,П,3,2,2,Челябинск,501.570,...,"−13,4","18,2","17,6","−49,9","39,9",429.0,1126.0,4.09,996.0,13.41


# Плотность

In [38]:
new_df['Плотность'] = new_df['POP_TOTAL'] / new_df['sqr_km']

# Saving

In [39]:
np.sum(new_df.isna())

lower_city                                      0
POP_TOTAL                                       0
2017 год                                        1
2018 год                                        1
2019 год                                        1
2017_quality                                    1
2018_quality                                    1
2019_quality                                    1
city                                            0
sqr_km                                          0
habbit_rate                                    12
perc_more_100k                                 12
percent_less_15k                               12
most_common                                    12
Средняя годовая температура, °C                12
Температура самого холодного месяца, °C        12
Температура трёх самых холодных месяцев, °C    12
Температура самого тёплого месяца, °C          12
Температура трёх самых тёплых месяцев, °C      12
Абсолютный минимум,                            12


In [40]:
new_df = new_df.drop('lower_city', axis=1)

In [42]:
new_df.to_csv('new_data/merged_v1_3.csv')

In [40]:
new_df

,POP_TOTAL,2017 год,2018 год,2019 год,2017_quality,2018_quality,2019_quality,city,sqr_km,habbit_rate,...,most_common,"Средняя годовая температура, °C","Температура самого холодного месяца, °C","Температура трёх самых холодных месяцев, °C","Температура самого тёплого месяца, °C","Температура трёх самых тёплых месяцев, °C","Абсолютный минимум,","Абсолютный максимум, °C","Среднее годовое количество осадков, мм",Плотность
0,1106635,Н,Н,Н,1,1,1,Уфа,700.000,"40,1",...,26–60,"3,8","−12,6","−11,6","19,6","18,3","−48,5","38,6",589.0,1580.907143
1,1567087,П,В,П,2,3,2,Новосибирск,506.670,"54,8",...,22–47,"1,8","−16,1","−14,7","19,2","17,5","−45,0","37,2",448.0,3092.914520
2,840785,П,П,П,2,2,2,Саратов,393.975,"79,8",...,17–37,"6,9","−8,1","−7,4","22,4","21,1","−37,3","40,9",476.0,2134.107494
3,309150,Н,Н,Н,1,1,1,Мурманск,154.600,"36,1",...,32–70,"0,6","−10,0","−9,3","12,9","11,1","−39,4","32,9",494.0,1999.676585
4,473895,Н,Н,Н,1,1,1,Чебоксары,110.000,NaN,...,NaN,"3,5","−12,9","−10,9","18,8","17,3","−44,3","39,9",499.0,4308.136364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,319550,Н,Н,Н,1,1,1,Орел,120.000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2662.916667
57,1017226,ОВ,ОВ,В,4,4,3,Красноярск,353.900,67,...,19–42,"1,6","−15,5","−13,9","18,7","16,8","−52,8","36,4",491.0,2874.331732
58,642263,Н,Н,П,1,1,2,Ульяновск,300.000,"73,6",...,18–36,5,"−10,3","−9,5","20,2","18,8","−40,0","39,3",470.0,2140.876667
59,5131942,П,Н,Н,2,1,1,Санкт-Петербург,1439.000,"46,2",...,35–76,"5,8","−5,8","−5,0","18,8","17,2","−35,9","37,1",662.0,3566.325226
